# Data preprocessing

### Libraries import

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)

### Data import and connecting

In [2]:
df_train = pd.read_hdf("../input/train_data.v2.h5")
df_test = pd.read_hdf("../input/test_data.v2.h5")

df = pd.concat([df_train, df_test])
df = df.reset_index(drop=True)
print(df.shape, df.shape)

(34180, 9) (34180, 9)


### How data looks like

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option("max_info_columns", 300)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34180 entries, 0 to 34179
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   geo_block    34180 non-null  object
 1   breadcrumbs  34180 non-null  object
 2   owner        34180 non-null  object
 3   params       34180 non-null  object
 4   date         34180 non-null  object
 5   user_block   34180 non-null  object
 6   id           34180 non-null  int64 
 7   price        22732 non-null  object
 8   params_en    34180 non-null  object
dtypes: int64(1), object(8)
memory usage: 2.3+ MB


In [5]:
df.head()

,geo_block,breadcrumbs,owner,params,date,user_block,id,price,params_en
0,"[г. Москва, Лианозово, г. Москва, Лианозово]","[Москва, Лианозово, м. Алтуфьево, МЦК Лихоборы]",[],"{'Охрана:': 'закрытая территория', 'Тип здания...","[22 апреля, 12, (+1 за сегодня), Обновлено 18 ...","[<div class=""block-user__name"">НДВ Недвижимост...",34995,5 402 084 ₽,"{'Security:': 'closed area', 'Building type:':..."
1,"[г. Москва, Ховрино, г. Москва, Ховрино]","[Москва, Ховрино, м. Ховрино, МЦК Коптево]",[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[10 мая, 2, (+1 за сегодня), Обновлено 16 мая]","[<div class=""block-user__name"">MR-Group</div>,...",15030,6 827 990 ₽,"{'Elevator:': 'yes', 'Building type:': 'Monoli..."
2,"[г. Москва, ул Лобачевского, г. Москва, ул Лоб...","[Москва, МЦК Кутузовская, ул. Лобачевского]","[ЖК ""Крылья"". Доп. скидка 5%!]","{'Лифт:': 'да', 'Охрана:': 'закрытая территори...","[20 марта, 12, (+2 за сегодня), Обновлено 18 мая]",[],37811,8 680 000 ₽,"{'Elevator:': 'yes', 'Security:': 'closed area..."
3,"[г. Москва, Ховрино, ул Дыбенко, г. Москва, Хо...","[Москва, Ховрино, м. Ховрино, МЦК Коптево, ул....",[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[29 апреля, 3, (+1 за сегодня), Обновлено 13 мая]","[<div class=""block-user__name"">MR-Group</div>,...",31685,8 624 435 ₽,"{'Elevator:': 'yes', 'Building type:': 'Monoli..."
4,"[г. Москва, Даниловский, г. Москва, Даниловский]","[Москва, Даниловский, м. Автозаводская, МЦК Ав...",[],"{'Общая площадь:': '141.1 м²', 'Дата публикаци...","[15 декабря 2018, 30, (+1 за сегодня), Обновле...","[<div class=""block-user__name"">Наталья Вдовиче...",56215,29 500 000 ₽,"{'Total area:': '141.1 м²', 'Date published:':..."


### Params

In [6]:
params_en = df["params_en"].apply(pd.Series)
params_en = params_en.fillna(-1)

if "Security:" not in df:
    df = pd.concat([df, params_en], axis=1)

### Extracting numerical values 

In [7]:
df[["Total area:", "Kitchen area:", "Ceiling height:", "Living room:"]].head()

,Total area:,Kitchen area:,Ceiling height:,Living room:
0,54.3 м²,-1,2.8 м,-1
1,38 м²,-1,3.15 м,-1
2,40 м²,-1,3 м,-1
3,43 м²,-1,3.15 м,-1
4,141.1 м²,-1,-1,-1


In [8]:
def parse_area(val):
    if isinstance(val, int): return val
    if isinstance(val, float): return val
    
    return float(val.split("м")[0].replace(" ", ""))

In [9]:
df["area"] = df["Total area:"].map(parse_area)
df["kitchen_area"] = df["Kitchen area:"].map(parse_area)
df["ceiling_height"] = df["Ceiling height:"].map(parse_area)
df["living_room"] = df["Living room:"].map(parse_area)

In [10]:
df[["area", "kitchen_area", "ceiling_height", "living_room"]].head()

,area,kitchen_area,ceiling_height,living_room
0,54.3,-1.0,2.80,-1.0
1,38.0,-1.0,3.15,-1.0
2,40.0,-1.0,3.00,-1.0
3,43.0,-1.0,3.15,-1.0
4,141.1,-1.0,-1.00,-1.0


### User block

In [13]:
df["user_block"][0]

['<div class="block-user__name">НДВ Недвижимость</div>',
 '<div class="block-user__agency"></div>']

In [14]:
df['block_user_name'] = df['user_block'].map(lambda x: x[0].split('>')[1].split('<')[0] if len(x) == 2 else -1)
df['block_user_agency'] = df['user_block'].map(lambda x: x[1].split('>')[1].split('<')[0] if len(x) == 2 else -1)

### Split data 

In [15]:
df_train = df[ ~df["price"].isnull() ].copy()
df_test = df[ df["price"].isnull() ].copy()
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

### Price to float

In [16]:
df_train['price'].head()

0     5 402 084 ₽
1     6 827 990 ₽
2     8 680 000 ₽
3     8 624 435 ₽
4    29 500 000 ₽
Name: price, dtype: object

In [17]:
def parse_price(val):
    if isinstance(val, str): 
        if "₽" in val:
            val = val.split('₽')[0]
            
        val = val.replace(' ', '')
        return int(val) / 1000000
    
    return float(val)

In [18]:
df_train['price'] = df_train['price'].map(parse_price)

### Saving data

In [19]:
df_train.to_csv("../output/post_preprocessing_train_data.csv", index=False)
df_test.to_csv("../output/post_preprocessing_test_data.csv", index=False)